In [ ]:
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
import numpy as np
import pickle

# Carregar dados
df = pd.read_csv("../data/livros_vendidos.csv")

# Separar colunas categóricas e numéricas
cat_cols = ["genero", "autor", "editora"]
num_cols = ["preco"]

X = df[cat_cols + num_cols]
y = df["vendas"]

# Codificar categorias com inteiros
le_tema = LabelEncoder().fit(X["genero"])
le_autor = LabelEncoder().fit(X["autor"])
le_editora = LabelEncoder().fit(X["editora"])

X_nn = pd.DataFrame(
    {
        "genero": le_tema.transform(X["genero"]),
        "autor": le_autor.transform(X["autor"]),
        "editora": le_editora.transform(X["editora"]),
        "preco": X["preco"],
    }
)

# Criar modelo com embeddings simples
input_tema = tf.keras.Input(shape=(1,))
input_autor = tf.keras.Input(shape=(1,))
input_editora = tf.keras.Input(shape=(1,))
input_preco = tf.keras.Input(shape=(1,))

emb_tema = tf.keras.layers.Embedding(len(le_tema.classes_), 8)(input_tema)
emb_autor = tf.keras.layers.Embedding(len(le_autor.classes_), 8)(input_autor)
emb_editora = tf.keras.layers.Embedding(len(le_editora.classes_), 8)(input_editora)

flat_tema = tf.keras.layers.Flatten()(emb_tema)
flat_autor = tf.keras.layers.Flatten()(emb_autor)
flat_editora = tf.keras.layers.Flatten()(emb_editora)

x = tf.keras.layers.Concatenate()([flat_tema, flat_autor, flat_editora, input_preco])
x = tf.keras.layers.Dense(32, activation="relu")(x)
output = tf.keras.layers.Dense(1)(x)

model_nn = tf.keras.Model(inputs=[input_tema, input_autor, input_editora, input_preco], outputs=output)
model_nn.compile(optimizer="adam", loss="mse")

# Treinar (poucas épocas só para exemplo)
model_nn.fit([X_nn["genero"], X_nn["autor"], X_nn["editora"], X_nn["preco"]], y, epochs=100, verbose=0)

# Salvar modelo
model_nn.save("_5_modelo_vendas.h5")

In [6]:
df

,genero,autor,preco,editora,vendas
0,Ciência,Rafael Gomes,125.72,Editora E,32850
1,Ficção,Carla Dias,38.29,Editora A,7325
2,Romance,Maria Oliveira,25.42,Editora C,1186
3,Aventura,João Silva,7.99,Editora E,581
4,Infantil,Pedro Martins,21.23,Editora E,1395
...,...,...,...,...,...
9995,Biografia,Thiago Fernandes,47.73,Editora D,30479
9996,Tecnologia,Sofia Almeida,101.37,Editora D,289
9997,Autoajuda,Maria Oliveira,36.39,Editora F,10191
9998,Ficção,Carla Dias,42.76,Editora E,235


In [ ]:
with open("_5_le_tema.pkl", "wb") as f:
    pickle.dump(le_tema, f)
with open("_5_le_autor.pkl", "wb") as f:
    pickle.dump(le_autor, f)
with open("_5_le_editora.pkl", "wb") as f:
    pickle.dump(le_editora, f)

# Novos livros para previsão
novos = pd.DataFrame(
    {
        "genero": ["Ficção"],
        "autor": ["Rafael Gomes"],
        "editora": ["Editora C"],
        "preco": [30],
    }
)

novo_livro_nn = pd.DataFrame(
    {
        "genero": le_tema.transform(novos["genero"]),
        "autor": le_autor.transform(novos["autor"]),
        "editora": le_editora.transform(novos["editora"]),
        "preco": novos["preco"],
    }
)

vendas_previstas = model_nn.predict(
    [novo_livro_nn["genero"], novo_livro_nn["autor"], novo_livro_nn["editora"], novo_livro_nn["preco"]]
)

print(f"Rede Neural Keras → {vendas_previstas}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
Rede Neural Keras → [[3127.1428]]


In [11]:
melhor_venda = -np.inf
melhor_config = None

for genero in le_tema.classes_:
    for autor in le_autor.classes_:
        for editora in le_editora.classes_:
            for preco in np.arange(10, 100, 1):  # testar preços de 10 a 100
                # codificar categorias
                tema_cod = le_tema.transform([genero])
                autor_cod = le_autor.transform([autor])
                editora_cod = le_editora.transform([editora])

                pred = model_nn.predict(
                    [
                        np.array(tema_cod).reshape(-1, 1),
                        np.array(autor_cod).reshape(-1, 1),
                        np.array(editora_cod).reshape(-1, 1),
                        np.array([[preco]]),
                    ],
                    verbose=0,
                )

                if pred > melhor_venda:
                    melhor_venda = pred
                    melhor_config = (genero, autor, editora, preco)

print("Melhor combinação:", melhor_config)
print("Vendas previstas:", melhor_venda)

Melhor combinação: ('Aventura', 'Lucas Lima', 'Editora D', np.int64(99))
Vendas previstas: [[4913.877]]
